# COMP47670 - Data Science in Python (MD)
NAME - SHAURYA GOGIA (19200891)

## Introduction


>**Data is collected from Zomato**. Zomato is an Indian restaurant aggregator and food delivery start up founded by Deepinder Goyal in 2008.

> As of 2019, the service is available in 24 countries and in more than 10,000 cities. 

> Zomato provides information, menus and user-reviews of restaurants, and also has food delivery options from partner restaurants in select cities. They provide restaurants details which offer variety of cuisines and have special highlights (Bar, Wifi, Pet friendly, Smoking area, Indoor/Outdoor seating area)

> This API was choosen to extract meaningful information about different restaurants in 15 countries. Restaurant details are extracted for capital city of these 15 countries. 

> (Source - https://en.wikipedia.org/wiki/Zomato)

## Dataset Description 

> Data is collected from Zomato API (https://developers.zomato.com/documentation).

Data is collected from 3 API's -
*   To get City ID of chosen cities, **/cities Zomato API** is used. The city IDs are used in the APIs ahead.
*   For each city, **/search Zomato API** is used to get the essential restaurant details in that city. As per the API restrictions, details of 100 restaurants for each city are extracted.   
*   For each restaurant, **/reviews Zomato API** is used to extract reviews. As per the API restrictions, upto 10 reviews are extracted for each restaurant.

Following Restaurant Details attributes are used - 

1. **id -**Unique ID of restaurant
2. **name -** Restaurant name
3. **city -** Restuarant city
4. **cuisines -** List of cuisines offered
5. **timings -** Restaurant timings
6. **average_cost_for_two -** Average cost for 2 people to have food in that restaurant
7. **price_range -** Price range compared to respective city
8. **highlights -** Additional perks offered
9. **aggregate_rating -** Aggregate ratings 
10. **all_reviews_count -** Total number of reviews provided 
11. **photo_count -** Total photos added
12. **has_online_delivery -** Whether restuarant has Online Delivery avaialable
13. **store_type -** Type of Store
14. **include_bogo_offers -** Has offers  
15. **is_table_reservation_supported -** Has the option of reserving table 


*   For each restaurant **/reviews Zomato API** is used to get restaurant reviews 

Following Review Details attributes are used -

1. **id -** Unique ID of restaurant
2. **name -** Restaurant name
3. **city -** Restuarant city
4. **rating -** Rating provided in the individual review
5. **review_text -** Text provided in review
6. **review_time_friendly -** Time at which review was provided 
7. **likes -** Total likes provided on the particular review
8. **name -** Name of reviewer
9. **foodie_level -** Zomato has set some levels for foodie (Example - Super foodie)


## Importing relevant packages

In [0]:
import numpy as np
import pandas as pd
import requests
import csv, time
from collections import OrderedDict
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS
import seaborn as sns

## Data Gathering and Collection

Data is gathered using the below keys - 

In [0]:
# API Key used
api_key = "8b437481731f34398dfb8a546b600eea"
user_key = api_key
header = {"user-key": user_key, "Accept": "application/json"}

In [0]:
# Function to convert list object into string (to be used later)
def listToString(lst):
  str1 = ""
  for i in range(0,len(lst)):
    if i == len(lst) - 1:
      str1 += lst[i]
    else:
      str1 += lst[i] + ", "
  return str1

In [0]:
''' Function to get the essential restaurant details from the /search Zomato API. '''

def get_restaurant_details(response, city, file1, file2, col_names1, col_names2):
  for entry in response:
    print("restaurant_id: " + str(entry['restaurant']['id']))
    entry_file1 = {}
    for key in col_names1:
      if key == 'highlights':
        entry_file1[key] = listToString(entry['restaurant']['highlights'])
      elif key == 'city':
        entry_file1[key] = city
      elif key in entry['restaurant'].keys():
        entry_file1[key] = entry['restaurant'][key]
      elif key in entry['restaurant']['location'].keys():
        entry_file1[key] = entry['restaurant']['location'][key]
      elif key in entry['restaurant']['user_rating'].keys():
        entry_file1[key] = entry['restaurant']['user_rating'][key]
      else:
        entry_file1[key] = ''
    write_row(file1, entry_file1.values())
    restaurant_id = entry['restaurant']['id']
    restaurant_name = entry['restaurant']['name']
    start = 0
    for i in range(0,2):
      url_reviews = "https://developers.zomato.com/api/v2.1/reviews?res_id=" + str(restaurant_id) + "&start=" + str(start)
      print(url_reviews)
      time.sleep(2)
      response_reviews = requests.get(url_reviews, headers = header).json()
      get_review_details(response_reviews['user_reviews'], restaurant_id, restaurant_name, city, file2, col_names2)
      start += 5

In [0]:
'''Function to get the essential review details from the /review API.'''

def get_review_details(response, res_id, res_name, city, file, col_names):
  for entry in response:
    print("review id: " + str(entry['review']['id']) + ", res_id: " + str(res_id))
    entry_file = {}
    for key in col_names:
      if key == 'city':
        entry_file[key] = city
      elif key == 'id':
        entry_file[key] = res_id
      elif key == 'res_name':
        entry_file[key] = res_name
      elif key in entry['review'].keys():
        entry_file[key] = entry['review'][key]
      elif key in entry['review']['user'].keys():
        entry_file[key] = entry['review']['user'][key]
      else:
        entry_file[key] = ''
    write_row(file, entry_file.values())

In [0]:
''' Function to create a file with the respective column names
'''
def create_file(file, col_names): 
  with open(file, "w", newline='') as write_file: #as in get_and_write_data function 
    f = csv.writer(write_file)
    f.writerow(col_names)
  write_file.close()

In [0]:
''' First function to start writing the results from the API into files.
    It invokes the /search Zomato API to get restaurant details.
    It then calls 'get_restaurant_details' function to get essential restaurant details.
'''
def write_file(city_ids, city_map, file1, file2, col_names1, col_names2):
  for city in city_ids:
    print("Current city : ", city_map[city_ids])
    start = 0
    for i in range(0,5):
      url_restaurants = "https://developers.zomato.com/api/v2.1/search?entity_id=" + str(city_ids) + "&entity_type=city&start=" + str(start) + "&count=20"
      print(url_restaurants)
      time.sleep(2)
      response_restaurants = requests.get(url_restaurants, headers = header).json()
      restaurant_numbers.add(city_map[city_ids] + "," + str(response_restaurants['results_found']))
      get_restaurant_details(response_restaurants['restaurants'], city_map[city_ids], file1, file2, col_names1, col_names2)
      start += 20

In [0]:
''' Function to write the file row by row.
    It is called from the 'get_restaurant_details' and 'get_review_details' functions.
'''
def write_row(file, response):
  with open(file, "a", newline='') as write_file:
    f = csv.writer(write_file)
    f.writerow(response)
  write_file.close()

In [0]:
''' All the required variables are initialised with the values
'''
# API names for the essential columns
col_names_response_restaurant = ["id", "name", "city", "cuisines", "timings", "average_cost_for_two", "price_range", "highlights", "aggregate_rating", "all_reviews_count", "photo_count", "has_online_delivery", "store_type", "include_bogo_offers", "is_table_reservation_supported"]
# Excel essential column names
col_names_restaurant_details = ["Restaurant ID", "Restaurant Name", "City", "Cuisines", "Timings", "Average Cost for Two", "Price Range", "Highlights", "Aggregate Rating", "Reviews Count", "Photos Count", "Online Delivery", "Store Type", "Bogo Offers", "Table Reservation"]
# API names for the essential columns
col_names_response_review = ["id", "res_name", "city", "rating", "review_text", "review_time_friendly", "likes", "name", "foodie_level"]
# Excel essential column names
col_names_review_details = ["Restaurant ID", "Restaurant Name", "City", "Restaurant Rating", "Review Text", "Review Time", "Likes", "User Name", "User Foodie Level"]

# Cities for which Zomato data is to be collected
cities = ["Dublin", "Canberra", "Ottawa", "Prague", "Delhi", "Rome", "Wellington", "Warsaw", "Lisbon", "Singapore", "Abu Dhabi", "London", "Washington", "Edinburgh", "Belfast"]

# A dictionary/map to store cities as key and their respective countries as values
country_city_dict = {"Dublin": "Republic of Ireland", "Canberra": "Australia", "Ottawa": "Canada", "Prague": "Czech Republic", "Delhi": "India", "Rome": "Italy", "Wellington": "New Zealand", "Warsaw": "Poland", "Lisbon": "Portugal", "Singapore": "Singapore", "Abu Dhabi": "United Arab Emirates", "London": "England", "Washington": "United States of America", "Edinburgh": "Sctoland", "Belfast": "Northern Ireland"}

# Initialized a set to store count of restaurants. Entries will be added in the 'get_restaurant_details' function.
restaurant_numbers = set()

In [0]:
''' Calls the /cities Zomato API for each city in the 'cities' list to get the respective city ID.
'''
# Initialized a list to store the city IDs
city_ids = []
# Initialised a dictionary to store city ID as key and its respective city as value
city_id_city_map = {}

# Iterates through every city in the 'cities' list and calls the /cities Zomato API
for city in cities:
    url = "https://developers.zomato.com/api/v2.1/cities?q=" + city
    city_request = requests.get(url, headers = header)
    response = city_request.json()
    city_id = response['location_suggestions'][0]['id']
    city_ids.append(city_id)
    city_id_city_map[city_id] = city

Creating 2 files for restaurant details and reveiw details 

In [0]:
''' Calls the functions to start gathering the data.
    First files 'restaurant_details.csv' and 'review_details.csv' are created.
    Then the 'write_file' function is called to start writing results into the files
'''
create_file("restaurant_details.csv", col_names_restaurant_details)
create_file("review_details.csv", col_names_review_details)
write_file(city_ids, city_id_city_map, "restaurant_details.csv", "review_details.csv", col_names_response_restaurant, col_names_response_review)

Using the above code, 2 files have been created -


*   restaurant_details.csv
*   review_details.csv



## Reading "restaurant_details.csv" file

In [0]:
#Created to handle any missing values and filling them afterwards
missing_values = ['{}', '', 'nan'] 
restaurant_details = pd.read_csv("restaurant_details.csv", na_values = missing_values)

#Analyzing restaurant dataset
restaurant_details

## Reading "review_details.csv" file

In [0]:
# Created to handle any missing values and filling them afterwards
missing_values = ['{}', '', 'nan'] 
review_details = pd.read_csv("review_details.csv", na_values = missing_values)

#Analyzing reviews dataset
review_details

## Essential code (present above which was used for collecting data) to run the below steps

Needed only if data gathering has been skipped

In [0]:
# Function to convert list object into string
def listToString(lst):
  str1 = ""
  for i in range(0,len(lst)):
    if i == len(lst) - 1:
      str1 += lst[i]
    else:
      str1 += lst[i] + ", "
  return str1

In [0]:
''' All the required variables are initialised with the values
'''
# API names for the essential columns
col_names_response_restaurant = ["id", "name", "city", "cuisines", "timings", "average_cost_for_two", "price_range", "highlights", "aggregate_rating", "all_reviews_count", "photo_count", "has_online_delivery", "store_type", "include_bogo_offers", "is_table_reservation_supported"]
# Excel essential column names
col_names_restaurant_details = ["Restaurant ID", "Restaurant Name", "City", "Cuisines", "Timings", "Average Cost for Two", "Price Range", "Highlights", "Aggregate Rating", "Reviews Count", "Photos Count", "Online Delivery", "Store Type", "Bogo Offers", "Table Reservation"]
# API names for the essential columns
col_names_response_review = ["id", "res_name", "city", "rating", "review_text", "review_time_friendly", "likes", "name", "foodie_level"]
# Excel essential column names
col_names_review_details = ["Restaurant ID", "Restaurant Name", "City", "Restaurant Rating", "Review Text", "Review Time", "Likes", "User Name", "User Foodie Level"]

# Cities for which Zomato data is to be collected
cities = ["Dublin", "Canberra", "Ottawa", "Prague", "Delhi", "Rome", "Wellington", "Warsaw", "Lisbon", "Singapore", "Abu Dhabi", "London", "Washington", "Edinburgh", "Belfast"]

# A dictionary/map to store cities as key and their respective countries as values
country_city_dict = {"Dublin": "Republic of Ireland", "Canberra": "Australia", "Ottawa": "Canada", "Prague": "Czech Republic", "Delhi": "India", "Rome": "Italy", "Wellington": "New Zealand", "Warsaw": "Poland", "Lisbon": "Portugal", "Singapore": "Singapore", "Abu Dhabi": "United Arab Emirates", "London": "England", "Washington": "United States of America", "Edinburgh": "Sctoland", "Belfast": "Northern Ireland"}

# Initialized a set to store count of restaurants. Entries will be added in the 'get_restaurant_details' function.
restaurant_numbers = set()

In [0]:
city_id_city_map = {91: 'Dublin', 313: 'Canberra', 295: 'Ottawa', 84: 'Prague', 1: 'Delhi', 257: 'Rome', 71: 'Wellington', 109: 'Warsaw', 82: 'Lisbon', 52: 'Singapore', 57: 'Abu Dhabi', 61: 'London', 283: 'Washington', 76: 'Edinburgh', 320: 'Belfast'}
city_ids = [91, 313, 295, 84, 1, 257, 71, 109, 82, 52, 57, 61, 283, 76, 320]

In [0]:
restaurant_numbers = {'Abu Dhabi,5171',
 'Belfast,1080',
 'Canberra,1060',
 'Delhi,36711',
 'Dublin,3212',
 'Edinburgh,2299',
 'Lisbon,14209',
 'London,28708',
 'Ottawa,2641',
 'Prague,6412',
 'Rome,10111',
 'Singapore,13041',
 'Warsaw,4134',
 'Washington,3065',
 'Wellington,1961'}

## Pre-processing restuarant details and review details

### Analyzing data before preprocessing is applied 


In [0]:
#Analyzing unique values in each column in restaurant details
for col in restaurant_details.columns:
  print(col + ":")
  print(restaurant_details[col].nunique())

**Observations from above results**

- Attribute **'Store type'** has no unique value 
- Attribute **'Bogo Offers'** has 1 unique value.

Both these are not informative enough. Hence they have been removed in the next step

In [0]:
#Analyzing unique values in each column in review details
for col in review_details.columns:
  print(col + ":")
  print(review_details[col].nunique())

### Data Cleaning

**Removing columns that are not informative**

In [0]:
# Removing columns 'Store Type' and 'Bogo Offers' since for every restaurant their values are same
restaurant_details.drop(columns = ['Store Type', 'Bogo Offers'], inplace = True)

**Handling missing and inconsistent data**

In ***restaurant_details***, there are 2 attributes that are important and have missing/inconsistent data -

*   **Average cost for 2** [Inconsistent values 0/275000017 ] 
*   **Cuisines** [Has missing values] 

Initially,  Inconsistent values are first converted to NA. Missing data have been handled by replacing them with 'Not specified'.

Later, 
*   For Average cost for 2, **NA is filled with the mean value for 'Average Cost for Two (in Euros)' of repective cities**. 
For example, if Delhi has a mean value for 'Average cost for two' as 400, the restaurants that have 'Average cost for two' as 0/25000017, is replaced by the mean value.

*  For Cuisines, each city's most popular cuisine is determined and **the'Not specified' cuisine is replaced by respective city's most popular cuisine**.
For example, if Dublin has the most popular cuisine as 'European', the resturants that have no cuisine specified is replaced by 'European'

In ***review_details***, missing data is replaced by NA





In [0]:
# Filling NA values with 'Not specified'
restaurant_details.fillna("Not Specified", inplace = True)

**Replacing inconsistent values in 'Average Cost for Two'**

In [0]:
# Replacing the values 0 and 25000017 (some random incorrect value) for 'Average Cost for Two' by NaN and filling it with mean 'Average Cost for Two' of repective cities
restaurant_details['Average Cost for Two'] = restaurant_details['Average Cost for Two'].replace([0, 25000017], np.nan)
restaurant_details['Average Cost for Two'] = restaurant_details.groupby("City")['Average Cost for Two'].transform(lambda x: x.fillna(int(round(x.mean()))))

**Replacing missing values in 'Cuisines'**

In [0]:
# Creating a set of cuisines which would be used in visualizations ahead
cuisines_set_with_spaces = set()
i = 0
for cuisine in restaurant_details['Cuisines']:
  cuisines_set_with_spaces.update(cuisine.split(','))

In [0]:
# Creating another set of cuisines by trimming the extra whitespaces
cuisines_set = set()
for entry in cuisines_set_with_spaces:
  entry = entry.strip()
  cuisines_set.add(entry)

cuisines_set

In [0]:
# Initialised a dictionary with each city as key and the values being count of each cuisine in that city
cuisine_city_dict = {"Dublin": {}, "Canberra": {}, "Ottawa": {}, "Prague": {}, "Delhi": {}, "Rome": {}, "Wellington": {}, "Warsaw": {}, "Lisbon": {}, "Singapore": {}, "Abu Dhabi": {}, "London": {}, "Washington": {}, "Edinburgh": {}, "Belfast": {}}

In [0]:
# Loops through each cuisine in the 'cuisines_set' to count the number of presence of that cuisine in each city
for cuisine in cuisines_set:
  data_cuisine = restaurant_details[restaurant_details['Cuisines'].str.contains(cuisine)]
  groupby_city = data_cuisine.groupby('City').count()['Restaurant ID']
  for index, num in groupby_city.items():
    cuisine_city_dict[index][cuisine] = num

cuisine_city_dict

In [0]:
# Replacing 'Not specified' cuisine by respective city's most popular cuisine
restaurant_details['Cuisines'] = restaurant_details['Cuisines'].str.replace('Not specified', max(cuisine_city_dict['Washington'], key = cuisine_city_dict['Washington'].get))

**Replacing missing data in review_details**

In [0]:
# Filling NA values with 'Not specified' 
review_details.fillna("Not Specified", inplace = True)

### Data Transformation 


Following data transformations have been applied for restaurant_details - 

*   **'Average cost for Two'** has cost values in different currencies, they are converted to Euros by applying the associated conversion rates
*   Some restaurants have **Aggregate ratings** and no **'Reviews count'**. There is no point of having Aggregate ratings as they will not be user provided. **Aggregate rating** values are hence replaced by 0 if there are no reviews (Review count = 0)
*   For attribute **'Online Delivery'** 0 is replaced by 'No' and 1 by 'Yes' 
*  For attribute **'Table reservation'** 0 is replaced by 'No' and 1 by 'Yes'

Following data transformations have been applied for review_details - 

*  **Review time** has been modified in a way that time is in the form of number of years ago
For example, if **'Review time'** is 20 July 2012, it is converted to 8 years ago


In [0]:
''' Function to convert cost values into Euros respective to each city and the associated conversion rate.
'''
def change_to_euros(row):
  if row['City'] == 'Canberra':
    row['Average Cost for Two'] *= 0.55
  if row['City'] == 'Ottawa':
    row['Average Cost for Two'] *= 0.64
  if row['City'] == 'Prague':
    row['Average Cost for Two'] *= 0.036
  if row['City'] == 'Delhi':
    row['Average Cost for Two'] *= 0.012
  if row['City'] == 'Wellington':
    row['Average Cost for Two'] *= 0.54
  if row['City'] == 'Warsaw':
    row['Average Cost for Two'] *= 0.22
  if row['City'] == 'Singapore':
    row['Average Cost for Two'] *= 0.63
  if row['City'] == 'Abu Dhabi':
    row['Average Cost for Two'] *= 0.24
  if row['City'] == 'Washington':
    row['Average Cost for Two'] *= 0.90
  if row['City'] == 'London' or row['City'] == 'Edinburgh' or row['City'] == 'Belfast':
    row['Average Cost for Two'] *= 1.12
  return int(round(row['Average Cost for Two']))

In [0]:
# Converting values of 'Average Cost for Two (in Euros)' into price in Euros
restaurant_details['Average Cost for Two (in Euros)'] = restaurant_details.apply(lambda row : change_to_euros(row), axis = 1)
restaurant_details.drop(columns = ['Average Cost for Two'])
restaurant_details['Average Cost for Two (in Euros)']

In [0]:
# Replacing Aggregate Rating values by 0 if there are no reviews i.e. Reviews Count = 0 for them
idxs = restaurant_details[(restaurant_details['Reviews Count'] == 0) & (restaurant_details['Aggregate Rating'] > 0)]['Aggregate Rating'].keys()
for idx in idxs:
  restaurant_details['Aggregate Rating'][idx] = 0
#restaurant_details[(restaurant_details['Reviews Count'] == 0) & (restaurant_details['Avg'] > 0)]

In [0]:
# Replacing 0 as 'No' and 1 as 'Yes' for column 'Online Delivery'
restaurant_details['Online Delivery'] = restaurant_details['Online Delivery'].apply(str)
restaurant_details['Online Delivery'] = restaurant_details['Online Delivery'].str.replace('0', 'No')
restaurant_details['Online Delivery'] = restaurant_details['Online Delivery'].str.replace('1', 'Yes')

# Replacing 0 as 'No' and 1 as 'Yes' for column 'Table Reservation'
restaurant_details['Table Reservation'] = restaurant_details['Table Reservation'].apply(str)
restaurant_details['Table Reservation'] = restaurant_details['Table Reservation'].str.replace('0', 'No')
restaurant_details['Table Reservation'] = restaurant_details['Table Reservation'].str.replace('1', 'Yes')

restaurant_details

In [0]:
#Data transformation
''' Function to modify review time such that time is in the form of number of years ago
'''
def modify_review_time(row):
  if row['Review Time'].find('2012') >= 0:
    row['Review Time'] = "8 years ago"
  if row['Review Time'].find('2013') >= 0:
    row['Review Time'] = "7 years ago"
  if row['Review Time'].find('2014') >= 0:
    row['Review Time'] = "6 years ago"
  if row['Review Time'].find('2015') >= 0:
    row['Review Time'] = "5 years ago"
  if row['Review Time'].find('2016') >= 0:
    row['Review Time'] = "4 years ago"
  if row['Review Time'].find('2017') >= 0:
    row['Review Time'] = "3 years ago"
  if row['Review Time'].find('2018') >= 0:
    row['Review Time'] = "2 years ago"
  if row['Review Time'].find('2019') >= 0:
    row['Review Time'] = "1 year ago"
  return row['Review Time']

In [0]:
# Review Time column values modified according to the above function 
review_details['Review Time'] = review_details.apply(lambda row : modify_review_time(row), axis = 1)
review_details

### Feature Engineering

Following features have been extracted -


1. **Payment Modes** has been extracted from **'Highlights'** column and created as a new column
2. **Takeway Available** has been extracted from **'Highlights'** column and created as a new column
3. **Serves Alcohol** has been extracted from **'Highlights'** column and created as a new column
4. **Vegetarian Friendly** has been extracted from **'Highlights'** column and created as a new column
5. **Seating Arrangement** has been extracted from **'Highlights'** column and created as a new column
6. **Special Features includes 'Has Early Bird Menu', 'Wifi', 'Nightlife' and 'Pet Friendly'** has been extracted from **'Highlights'** column and created as a new column - Special features
7. Dropping column **'Highlights'** since all required features are extracted







In [0]:
''' Function to extract 'Payment Modes' from the 'Highlights' column
'''
def make_payment_mode_string(row):
  ret_str = ""
  if row['Highlights'].find('Credit Card') >= 0:
    ret_str += "Credit Card,"
  if row['Highlights'].find('Debit Card') >= 0:
    ret_str += "Debit Card,"
  if row['Highlights'].find('Cash') >= 0:
    ret_str += "Cash,"
  if row['Highlights'].find('Digital Payments Accepted') >= 0:
    ret_str += "Digital Payments Accepted,"
  if ret_str == "":
    return "Not specified"
  else:
    return ret_str[0:len(ret_str) - 1]

In [0]:
# 1. Payment Modes column created by checking Highlights column
restaurant_details['Payment Modes'] = restaurant_details.apply(lambda row : make_payment_mode_string(row), axis = 1)

In [0]:
''' Function to extract 'Takeaway Available' information from the 'Highlights' column
'''
def make_takeaway_string(row):
  ret_str = ""
  if row['Highlights'].find('Takeaway Available') >= 0:
    ret_str = "Yes"
  else:
    ret_str = "No"
  return ret_str

In [0]:
# 2. Takeaway Available column created by checking Highlights column
restaurant_details['Takeaway Available'] = restaurant_details.apply(lambda row : make_takeaway_string(row), axis = 1)

In [0]:
''' Function to extract 'Serves Alcohol' information from the 'Highlights' column
'''
def make_alcohol_string(row):
  ret_str = ""
  if row['Highlights'].find('Serves Alcohol') >= 0:
    ret_str = "Yes"
  else:
    ret_str = "No"
  return ret_str

In [0]:
# 3. Serves Alcohol column created by checking Highlights column
restaurant_details['Serves Alcohol'] = restaurant_details.apply(lambda row : make_alcohol_string(row), axis = 1)

In [0]:
''' Function to extract 'Vegetarian Friendly' information from the 'Highlights' column
'''
def make_vegetarian_string(row):
  ret_str = ""
  if row['Highlights'].find('Vegetarian Friendly') >= 0:
    ret_str = "Yes"
  else:
    ret_str = "No"
  return ret_str

In [0]:
# 4. Vegetarian Friendly column created by checking Highlights column
restaurant_details['Vegetarian Friendly'] = restaurant_details.apply(lambda row : make_vegetarian_string(row), axis = 1)

In [0]:
''' Function to extract 'Seating Arrangements' information from the 'Highlights' column
'''
def make_seating_string(row):
  ret_str = ""
  if row['Highlights'].find('Indoor Seating') >= 0:
    ret_str += "Indoor,"
  if row['Highlights'].find('Outdoor Seating') >= 0:
    ret_str += "Outdoor,"
  if ret_str == "":
    return "Not specified"
  else:
    return ret_str[0:len(ret_str) - 1]

In [0]:
# 5. Seating Arrangements column created by checking Highlights column
restaurant_details['Seating Arrangements'] = restaurant_details.apply(lambda row : make_seating_string(row), axis = 1)

In [0]:
''' Function to extract 'Special Features' information from the 'Highlights' column.
    Special Features includes 'Has Early Bird Menu', 'Wifi', 'Nightlife' and 'Pet Friendly'
'''
def make_special_features_string(row):
  ret_str = ""
  if row['Highlights'].find('Has Early Bird Menu') >= 0:
    ret_str += "Early Bird Menu,"
  if row['Highlights'].find('Wifi') >= 0:
    ret_str += "Wifi,"
  if row['Highlights'].find('Nightlife') >= 0:
    ret_str += "Nightlife,"
  if row['Highlights'].find('Pet Friendly') >= 0:
    ret_str += "Pet Friendly,"
  if ret_str == "":
    return "Not specified"
  else:
    return ret_str[0:len(ret_str) - 1]

In [0]:
# 6. Special Features column created by checking Highlights column
restaurant_details['Special Features'] = restaurant_details.apply(lambda row : make_special_features_string(row), axis = 1)

In [0]:
# 7. Dropping column Highlights since all required features are extracted
restaurant_details.drop(columns = 'Highlights', inplace = True)

In [0]:
restaurant_details

## Visualisation 

Initializing variables to be used later in visualizations 

In [0]:
font = {'family': 'serif',
        'weight': 'normal',
        'size': 13,
        'horizontalalignment': 'center'
        }

font_horizontal = {'family': 'serif',
        'weight': 'normal',
        'size': 13,
        'verticalalignment': 'center'
        }

colors = ['lightcoral', 'palegreen', 'chocolate', 'plum', 'turquoise', 'cornflowerblue', 'dimgrey', 'pink', 'wheat', 'salmon', 'mediumpurple', 'coral', 'mediumseagreen', 'khaki', 'lightsteelblue']

Below code is used to find most popular cuisine for each city and the corresponding number of restaurants offering that cuisine 
An ordered dictionary is created to retain the order in which the values are entered. 

**Key is of the form - [city,most popular cuisine]**

**Value is - [the number of restaurants]**

Key is taken in this form to prevent duplicate keys. For example, if there are 2 cities which have same 'Most popular cuisines', then instead of making 2 entries, it will overwrite the last one.


In [0]:
# Finding most popular cuisine for each city and keeping them in an ordered dictionary to retain the order in which values are entered
max_cuisine_dict = {}
for city in city_id_city_map.values():
  max_key = max(cuisine_city_dict[city], key = cuisine_city_dict[city].get)
  max_cuisine_dict[city + "," + max_key] = cuisine_city_dict[city][max_key]

max_cuisine_dict_ordered = OrderedDict(max_cuisine_dict)
max_cuisine_dict_ordered

In [0]:
# Code to get the highly rated and worst rated restaurants in all cities
highly_rated_restaurant_names = {}
worst_rated_restaurant_names = {}
for city in cities:
  highly_rated_restaurant_names[city] = restaurant_details[restaurant_details['City'] == city].sort_values(by = ['Aggregate Rating', 'Reviews Count'], ascending = False)['Restaurant Name'].iloc[0]
  worst_rated_restaurant_names[city] = restaurant_details[restaurant_details['City'] == city].sort_values(by = ['Aggregate Rating', 'Reviews Count'], ascending = True)['Restaurant Name'].iloc[0]

In [0]:
highly_rated_restaurant_names

In [0]:
worst_rated_restaurant_names

### Visualisation 1
Total number of restaurants in each city 

In [0]:
# Creating a dictionary from the set 'restaurant_numbers' which stores city as key and the count of restaurants as value
restaurant_numbers_dict = {}
for item in restaurant_numbers:
  splits = item.split(',')
  restaurant_numbers_dict[splits[0]] = int(splits[1])

# Restaurant Count in each city
fig = plt.figure(figsize = (11,8))
plt.bar(list(restaurant_numbers_dict.keys()), restaurant_numbers_dict.values(), color = colors)
plt.tight_layout()
for index, value in enumerate(restaurant_numbers_dict.values()):
    plt.text(index, value, str(value), fontdict = font)
plt.xlabel("Count of Restaurants")
plt.ylabel("Cities")
plt.xticks(rotation = 90)
plt.title("Number of Restaurants in Each City")
plt.show()

**Observations -**

**India's capital - Delhi has 36711 restaurants served by Zomato - highest in any country**
In Europe, London has the highest number of restaurants - 28708
Australian capital - Canberra has only 1060 restaurants followed by 1080 restaurants in Belfast

### Visualisation 2 -

Top 10 poular cuisines in Dublin based on number of restaurants offering that cuisine 


In [0]:
# Sorting the cuisines based on popularity in descending order and appending it to a list
cuisine_list = []
{cuisine_list.append(k+","+str(v)) for k, v in sorted(cuisine_city_dict['Dublin'].items(), key=lambda item: item[1], reverse=True)}

# Creating an ordered dictionary of cuisines of Dublin sorted on the basis of count of restaurants to retain the order in which values are entered
cuisine_list_dict = {}
for item in cuisine_list:
  items = item.split(",")
  cuisine_list_dict[items[0]] = int(items[1])
cuisine_list_dict_ordered = OrderedDict(cuisine_list_dict)

print(cuisine_list_dict_ordered)

#Plotting bar plot
labels = list(cuisine_list_dict_ordered.keys())[0:10]
sizes = list(cuisine_list_dict_ordered.values())[0:10]
explode = [0.1,0,0,0,0,0,0,0,0,0]
fig = plt.figure(figsize=(10,10))
plt.pie(sizes, startangle=90, autopct='%1.1f%%', labels=labels, explode = explode, shadow = True)
plt.axis('equal')
plt.title('Top 10 popular cuisines in Dublin based on number of restaurants offering that cuisine ')
plt.tight_layout()
plt.show()

**Observations -**

From the above bar chart, it is observed that **'European food' is the most popular cuisine in Dublin followed by 'Irish food' and 'Cafe'.**

***Around 21% restaurants in Dublin serve 'European food'***

### Visualisation 3
Mean 'Average cost for Two' for each city

In [0]:
# Mean 'Average Cost for Two (in Euros)'
mean_avg_cost_for_two = restaurant_details.groupby('City')['Average Cost for Two (in Euros)'].mean()

fig = plt.figure(figsize = (12,7))
mean_avg_cost_for_two = mean_avg_cost_for_two.sort_values()
xlab = list(mean_avg_cost_for_two.keys())
ylab = mean_avg_cost_for_two.values
plt.plot(xlab, ylab, marker = 'o')
plt.tight_layout()
for i,j in zip(xlab, ylab):
  plt.annotate(j, (i,j), textcoords="offset points", xytext=(0,10), ha='center')
plt.xlabel("Cities")
plt.ylabel("Mean Average Cost for Two (in Euros)")
plt.title("Mean Average Cost for Two (in Euros) for Each City")
plt.show()

**Observations -**

When currencies are converted in Euros, restaurants in Washington have the highest mean value for 'Average Cost for Two'- 80.96 Euros

Average cost for Two in Dublin is around 58 Euros.

Average cost for Two in Delhi is around 9 Euros


### Visualisation 4
Number of restaurants v/s different payment modes

In [0]:
''' Variables to count the occurences of different Payment Modes in all restaurants
    Payment Modes - 1. Credit Card
                    2. Debit Card
                    3. Cash
                    4. Digital Payments Accepted
'''
count_credit_card = restaurant_details[restaurant_details['Payment Modes'].str.contains('Credit Card')]
count_debit_card = restaurant_details[restaurant_details['Payment Modes'].str.contains('Debit Card')]
count_cash = restaurant_details[restaurant_details['Payment Modes'].str.contains('Cash')]
count_digital_payments = restaurant_details[restaurant_details['Payment Modes'].str.contains('Digital Payments Accepted')]

# Different Payment Modes
counts_payment_modes = [len(count_credit_card), len(count_debit_card), len(count_cash), len(count_digital_payments)]
payment_modes = ['Credit Card', 'Debit Card', 'Cash', 'Digital Payments']
fig = plt.figure(figsize = (5,5))
plt.bar(payment_modes, counts_payment_modes, color = colors)
plt.tight_layout()
for index, value in zip(range(0,len(counts_payment_modes)), counts_payment_modes):
    plt.annotate(value, (index, value), textcoords="offset points", xytext=(0,5), ha='center')
plt.xlabel("Payment Modes")
plt.ylabel("Count of Restaurants")
plt.title("Number of Restaurants V/S Different Payment Modes")
plt.show()

**Observations -**

### Visualisation 5
Total number of restaurants with online delivery service in cities'

In [0]:
# Count restuarants in each city with have 'Online Delivery' available
count_online_delivery = restaurant_details[restaurant_details['Online Delivery'] == 'Yes'].groupby('City').count()['Restaurant ID']

# Plotting bar chart
plt.barh(list(count_online_delivery.keys()), count_online_delivery.values, color = colors)
for index, value in enumerate(count_online_delivery.values):
    plt.text(value, index, str(value), fontdict = font_horizontal)
plt.xlabel("Count of Restaurants")
plt.ylabel("Cities")
plt.title("Number of Restaurants with Online Delivery Service in Cities")
plt.show()

**Observations -**

Out of 15 cities, only 2 cities have Online Delivery available - Delhi and Abu Dhabi.

**Out of 100 restaurants, 91 and 89 restaurants offer 'Online Delivery' in Delhi** **and Abu Dhabi respectively.**

### Visualisation 6

Total number of restaurants with 'Takeaway available' in each city

In [0]:
# 'Takeaway Available' count
count_takeaway_available = restaurant_details[restaurant_details['Takeaway Available'] == 'Yes'].groupby('City').count()['Restaurant ID']
fig = plt.figure(figsize = (8,7))
plt.bar(list(count_takeaway_available.keys()), count_takeaway_available.values, color = colors)
for index, value in enumerate(count_takeaway_available.values):
    plt.text(index, value, str(value), fontdict = font)
plt.xlabel("Cities")
plt.ylabel("Count of Restaurants")
plt.tight_layout()
plt.xticks(rotation = 90)
plt.title('Number of Restaurants with Takeaway Available in Cities')
plt.show()

**Observations-**

Out of 100 restaurants **in Prague, 94 are offering 'Takeaway service',** followed by 92 restaurants in Abu Dhabi

Only **6 restaurants have 'Takeaway service' in Belfast**

### Visualisation 7
Total number of restaurants offering Vegetarian Friendly menu in each city

In [0]:
# 'Vegetarian Friendly' count
count_vegetarian_friendly = restaurant_details[restaurant_details['Vegetarian Friendly'] == 'Yes'].groupby('City').count()['Restaurant ID']

fig = plt.figure(figsize = (10,7))
plt.barh(list(count_vegetarian_friendly.keys()), count_vegetarian_friendly.values, align = "center", color = colors)
plt.tight_layout()
for index, value in enumerate(count_vegetarian_friendly.values):
    plt.text(value, index, str(value), fontdict = font_horizontal)
plt.xlabel("Count of Restaurants")
plt.ylabel("Cities")
plt.title("Number of Restaurants Offering Vegetarian Friendly Menus in Each City")
plt.show()

**Observation -**

**Wellington has 80 restaurants which have Vegetarian Friendly** menus available.

Edinburgh/Canberra have only 5 restaurants offering Vegetarian Friendly menus

### Visualisation 8 
Total number of restaurants offering 'Table Reservation' in each city 

In [0]:
# 'Table Reservation' count
count_table_reservation = restaurant_details[restaurant_details['Table Reservation'] == 'Yes'].groupby('City').count()['Restaurant ID']

fig = plt.figure(figsize = (10,7))
plt.barh(list(count_table_reservation.keys()), count_table_reservation.values, edgecolor = 'black', color = colors)
plt.tight_layout()
for index, value in enumerate(count_table_reservation.values):
    plt.text(value, index, str(value), fontdict = font_horizontal)
plt.xlabel("Count of Restaurants")
plt.ylabel("Cities")
plt.title("Number of Restaurants with Table Reservation Service in Cities")
plt.show()

**Observations -**

**Delhi has 23 restuarnts which have the facility of 'Table Reservation' available** followed by 21 in Abu Dhabu.

Dublin has only 1 restaurant out of 100 which offers 'Table Reservation'

### Visualisation 9
Box plot for 'Average cost for Two'

In [0]:
data_for_boxplot = pd.DataFrame(columns = restaurant_details.columns)
for city in cities:
  data_for_boxplot = data_for_boxplot.append(restaurant_details[(restaurant_details['City'] == city) & (restaurant_details['Cuisines'].str.contains(list(max_cuisine_dict_ordered.keys())[cities.index(city)].split(",")[1]))])

data_for_boxplot

fig, axx = plt.subplots(figsize=(20,10))
sns.boxplot(x = 'City', y = 'Average Cost for Two (in Euros)', data = data_for_boxplot)

**Observations -**

1. The 'Average cost for Two' in Belfast and Canberra restaurants are observed to be less distributed. Since for Canberra, 100% results lie between 10-15 Euros and for Belfast, all the results lie in the same range
2. Dublin and Ottawa have similar kind of distributions (with the only excpetion of two outliers in Ottawa), almost half values lie at the median. 
3. Washington has highly distributed values with only few values lying in the first quartile. 

### Visualisation 10
Total number of restaurants satisfying a particular criteria in 5 cities

Criteria -
1. Aggregate rating >=4.7
2. Payment mode includes 'Credit card'
3. Should Serve Alcohol
4. Cities - 'London', 'Washington', 'Rome', 'Ottawa', 'Wellington'





In [0]:
count_fitered_restaurants = []
filtered_cities = ['London', 'Washington', 'Rome', 'Ottawa', 'Wellington']
for city in filtered_cities:
  results = restaurant_details[(restaurant_details['Aggregate Rating'] >= 4.7) & (restaurant_details['Payment Modes'].str.contains('Credit Card')) & (restaurant_details['Serves Alcohol'] == 'Yes') & (restaurant_details['City'] == city) & (restaurant_details['Cuisines'].str.contains(list(max_cuisine_dict_ordered.keys())[cities.index(city)].split(",")[1]))]
  count_fitered_restaurants.append(len(results))

fig = plt.figure(figsize = (5,5))
plt.bar(filtered_cities, count_fitered_restaurants, color = colors)
plt.tight_layout()
for index, value in zip(range(0,len(count_fitered_restaurants)), count_fitered_restaurants):
    plt.annotate(value, (index, value), textcoords="offset points", xytext=(0,5), ha='center')
plt.xlabel("Cities")
plt.ylabel("Count of Restaurants")
plt.title("Number of Restaurants with Specific Criteria in Different Cities")
plt.show()

**Observations -**

There are **7 restuarants in London offered by Zomato which have satisifed the following criteria** -

1. Aggregate rating >=4.7
2. Payment mode includes 'Credit card'
3. Should Serve Alcohol

Only 1 restaurants in Wellington has satisfied the above criteria



### Visualisation 11
Average cost v/s Aggregate rating

In [0]:
delhi_restaurants = restaurant_details[restaurant_details['City'] == 'Delhi']
ax = sns.regplot(x = 'Average Cost for Two (in Euros)', y = 'Aggregate Rating', data = delhi_restaurants)

**Observations -**
Particulary for Delhi, 'Aggregate rating' is correlated with 'Average cost for Two'. This means restaurants with higher prices have good ratings.

### Visualisation 12
Number of Restaurants with Aggregate Rating >= 4.5 for Each City

In [0]:
count_aggregate_ratings = {}
for city in cities:
  results = restaurant_details[(restaurant_details['Aggregate Rating'] >= 4.5) & (restaurant_details['City'] == city)]
  count_aggregate_ratings[city] = len(results)
  fig = plt.figure(figsize = (6,6))
plt.barh(list(count_aggregate_ratings.keys()), count_aggregate_ratings.values(), color = colors)
plt.tight_layout()
for index, value in enumerate(count_aggregate_ratings.values()):
    plt.text(value, index, str(value), fontdict = font_horizontal)
plt.xlabel("Count of Restaurants")
plt.ylabel("Cities")
plt.title("Number of Restaurants with Aggregate Rating >= 4.5 for Each City")
plt.show()

**Observations -**

From Visualisation 6, it is observed -

1. Delhi has 36711 restaurants 
2. London has 28708 restaurants
3. Canberra has only 1060 restaurants
4. Belfast has 1080 restaurants

From above bar chart, it is observed -
1. Out of 36711 restaurants in Delhi, Only 18 resturants have an Average Rating>=4.5
2. Out of 28708 restaurants in London, Only 74 resturants have an Average Rating>=4.5
3. Out of 1060 restaurants in Canberra, Only 11 resturants have an Average Rating>=4.5
4. Out of 1080 restaurants in Belfast, Only 2 resturants have an Average Rating>=4.5

### Visualisation 13
Word Cloud is created for reviews of Highly rated and worst rated restaurant in Delhi.
Both the API's data (Restaurant details and review details) is used together -
1. Using the restaurant details, highly and worst rated restaurants are determined by using 'Aggregate rating' attribute first.
2. There can be multiple restaurants which have the same value for 'Aggregate rating'. In that case, review count is considered. The one which has higher 'Review Count' is considered as Highly rated 
3. Using the restaurant id, Reviews_data is parsed and all the reviews correspoding to the id is collected.


In [0]:
#Removing stopwords
stopwords = set(STOPWORDS)
stopwords.add("Specified'".lower())
stopwords.add("'Not".lower())
for restaurant in listToString(list(review_details['Restaurant Name'])).split(','):
  if restaurant.find(' ') >= 0:
    for word in restaurant.split(' '):
      stopwords.add(word.lower().strip())
  else:
    stopwords.add(restaurant.lower().strip())

In [0]:
#Function to display WordCloud
def show_wordcloud(data, title = None):
  wordcloud = WordCloud(
    background_color='white',
    stopwords=stopwords,
    max_words=50,
    max_font_size=40,
    scale=3,
    random_state=1 # chosen at random by flipping a coin; it was heads
  ).generate(str(data).lower())

  print(wordcloud.words_)
  fig = plt.figure(1, figsize=(12, 12))
  plt.axis('off')
  if title: 
    fig.suptitle(title, fontsize=20)
    fig.subplots_adjust(top=2.3)

  plt.imshow(wordcloud)
  plt.show()

In [0]:
# Code to get the highly rated and the worst rated restaurants in Delhi
  highly_rated_restaurant_in_delhi = restaurant_details[restaurant_details['City'] == 'Delhi'].sort_values(by = ['Aggregate Rating', 'Reviews Count'], ascending = False)['Restaurant Name'].iloc[0]
  worst_rated_restaurant_in_delhi = restaurant_details[restaurant_details['City'] == 'Delhi'].sort_values(by = ['Aggregate Rating', 'Reviews Count'], ascending = True)['Restaurant Name'].iloc[0]


In [0]:
highly_rated_restaurant_in_delhi

In [0]:
show_wordcloud(list(review_details[(review_details['City'] == 'Delhi') & (review_details['Restaurant Name'] == highly_rated_restaurant_in_delhi)]['Review Text']), "Highly Rated Restaurant's Reviews in Dublin")

In [0]:
worst_rated_restaurant_in_delhi

In [0]:
show_wordcloud(list(review_details[(review_details['City'] == 'Delhi') & (review_details['Restaurant Name'] == worst_rated_restaurant_in_delhi)]['Review Text']), "Worst Rated Restaurant's Reviews in Dublin")

## Conclusion

Using the above analysis on Zomato's API, following observations have been made -
For Zomato staff
1. Delhi has the highest number of restaurants and Canberra has the lowest number of restaurants available in Zomato 
2. Only 2 cities have 'Online Delivery' option available - Abu Dhabi and Dubai
3. Prague has most restaurants offering takeaways. Belfast has very few restaurants offering takeaways
4. Wellington has many restaurants offering vegetarian menu, while Edinburgh and Canberra have very few restaurants offering vegetarian menu
5. Dublin has only 1 resturant out of 100 restaurants which offers Table reservation.
6. In Euros, average cost for 2 people is very high in Washington, Singapore and London. It's very low in Delhi [Reason can be low value of currency]
7. Most of the restaurants have credit card as a payment method.
8. Although Delhi has the highest number of restaurants, only 18 restuarants have an average rating >=4.5
9. In some way, Aggregate rating is correlated with Average cost for 2. This means restaurants with higher prices have good ratings.
10. Most popular cuisines in each of the 15 cities 

    * 'Abu Dhabi': 'Indian',
    * 'Belfast': 'British',
    * 'Canberra': 'Tea',
    * 'Delhi': 'Indian',
    * 'Dublin': 'European',
    * 'Edinburgh': 'Scottish',
    * 'Lisbon': "Italian",
    * 'London': 'British',
    * 'Ottawa': 'Canadian',
    * 'Prague': 'Czech',
    * 'Rome': 'Italian',
    * 'Singapore': 'American',
    * 'Warsaw': 'European',
    * 'Washington': 'American',
    * 'Wellington': 'Cafe Food'

11. Highly rated restaurants in each city 

    * 'Abu Dhabi': "Max's Restaurant",
    * 'Belfast': 'Mourne Seafood Bar',
    * 'Canberra': 'The Cupping Room',
    * 'Delhi': 'Ah So Yum',
    * 'Dublin': 'Brother Hubbard',
    * 'Edinburgh': 'Vietnam House Restaurant & Gallery',
    * 'Lisbon': "Forno d'Oro",
    * 'London': 'Duck & Waffle',
    * 'Ottawa': 'Fraser Café',
    * 'Prague': 'Café Imperial',
    * 'Rome': 'Grazia & Graziella',
    * 'Singapore': 'Lau Pa Sat',
    * 'Warsaw': 'Uki Uki',
    * 'Washington': 'Founding Farmers - D.C.',
    * 'Wellington': 'Ombra'

12. Worst rated restaurants in each city

    * 'Abu Dhabi': 'Pizza Hut',
    * 'Belfast': 'Wagamama',
    * 'Canberra': 'The Merchant',
    * 'Delhi': 'Sunil Momos.com',
    * 'Dublin': "Ravi's Kitchen",
    * 'Edinburgh': 'Noks Kitchen',
    * 'Lisbon': 'The Great American Disaster',
    * 'London': 'The Big Easy',
    * 'Ottawa': 'TacoLot',
    * 'Prague': 'Hadovka restaurant',
    * 'Rome': 'Bread & Break',
    * 'Singapore': 'Gastrosmiths',
    * 'Warsaw': 'Frida',
    * 'Washington': 'RPM Italian',
    * 'Wellington': 'Thunderbird Cafe'

Further analysis that can be done -

- This analysis can be further enhanced to include all cities for all countries. 

- For Zomato owners, further analysis on type of responses in each city can help them to expand their business model

- For travellers who are travelling to different countries, they can use the different visualisations to decide the type of cuisine and restaurants to choose, based on popularity and rating 


